In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# !pip install seaborn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import seaborn as sns
import re
import nltk
import os

In [10]:
df=pd.read_csv("G:\Codes\GenAI\Gen-Ai\Dataset\IMDB_Dataset.csv")

In [11]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
df=df.iloc[:10000]

In [13]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [14]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
df.duplicated().sum()

17

In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.duplicated().sum()

0

# Basic Preprocrssing

In [18]:
import re 
def remove_tags(text):
    return re.compile(r'<[^>]+>').sub('', text)

In [19]:
df['review']=df['review'].apply(remove_tags)

In [20]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
df['review']=df['review'].apply(lambda x: x.lower())

In [22]:
from nltk.corpus import stopwords
import nltk

sw_list=stopwords.words('english')

df['review']=df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw_list)]))

In [23]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
9995,"fun, entertaining movie wwii german spy (julie...",positive
9996,"give break. anyone say ""good hockey movie""? kn...",negative
9997,movie bad movie. watching endless series bad h...,negative
9998,"movie probably made entertain middle school, e...",negative


In [24]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [25]:
X=df.iloc[:,0:1]
y=df['sentiment']

In [26]:
print(X)
print(y)

                                                 review
0     one reviewers mentioned watching 1 oz episode ...
1     wonderful little production. filming technique...
2     thought wonderful way spend time hot summer we...
3     basically there's family little boy (jake) thi...
4     petter mattei's "love time money" visually stu...
...                                                 ...
9995  fun, entertaining movie wwii german spy (julie...
9996  give break. anyone say "good hockey movie"? kn...
9997  movie bad movie. watching endless series bad h...
9998  movie probably made entertain middle school, e...
9999  smashing film film-making. shows intense stran...

[9983 rows x 1 columns]
0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object


In [27]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)

In [30]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

print(X_train.shape)
print(X_test.shape)

(7986, 1)
(1997, 1)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [34]:
X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()

In [36]:
print(X_train_bow)
print(X_test_bow)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [37]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(X_train_bow,y_train)

GaussianNB()

# Word2vec

In [38]:
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train Word2Vec model
sentences = [review.split() for review in df['review']]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to transform reviews into vectors
def transform_to_vector(review, model):
    words = review.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Transform reviews in training and test sets
X_train_w2v = np.array([transform_to_vector(review, w2v_model) for review in X_train['review']])
X_test_w2v = np.array([transform_to_vector(review, w2v_model) for review in X_test['review']])

# Train Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=0)
rf_classifier.fit(X_train_w2v, y_train)

# Predict and evaluate
y_pred = rf_classifier.predict(X_test_w2v)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7175763645468203


# N-Gram

In [40]:
cv = CountVectorizer(ngram_range=(2,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.7676514772158237

# TF-IDF

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [42]:

X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [43]:

rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.8502754131196795

# Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LogisticRegression

# Prepare tagged documents
tagged_data = [TaggedDocument(words=review.split(), tags=[str(i)]) for i, review in enumerate(df['review'])]

# Train Doc2Vec model
d2v_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4, epochs=20)

# Transform reviews into vectors
X_train_d2v = np.array([d2v_model.infer_vector(review.split()) for review in X_train['review']])
X_test_d2v = np.array([d2v_model.infer_vector(review.split()) for review in X_test['review']])

# Train Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)
lr_classifier.fit(X_train_d2v, y_train)

# Predict and evaluate
y_pred_d2v = lr_classifier.predict(X_test_d2v)
accuracy_d2v = accuracy_score(y_test, y_pred_d2v)
print(f'Doc2Vec Accuracy: {accuracy_d2v}')



Doc2Vec Accuracy: 0.8217325988983475


In [47]:
print(np.array([d2v_model.infer_vector("This movie is great".split())]))

[[ 0.01684307  0.06060658  0.04006665  0.02827395  0.0066178  -0.02203297
  -0.08126394  0.18189168 -0.0224479  -0.02735722  0.02138779 -0.07525918
  -0.06104195 -0.01873214  0.06745962 -0.07115564  0.06366579 -0.05881545
  -0.00425324  0.00372993  0.05195734  0.0171929   0.05187379 -0.07127289
   0.02959352  0.05186051  0.0027818   0.02367237 -0.10198105  0.04645378
   0.02443234  0.07092364  0.01959173  0.04179678  0.00063328  0.10861904
  -0.06928372  0.05110504 -0.01963806 -0.02355274 -0.01694466 -0.00380962
   0.0345706  -0.02606351  0.05815877  0.00774618 -0.02048066 -0.06310249
   0.10655922 -0.0255561   0.05097194 -0.02460548 -0.01623863 -0.07455003
  -0.00902922 -0.00907496  0.01181822 -0.02783796 -0.01378989  0.00019224
   0.06737225  0.03394788  0.00906129  0.01257176 -0.09931628  0.0514859
   0.02152262  0.00824291 -0.05647143  0.07716727 -0.08836313  0.10234925
   0.03902544 -0.017815   -0.01436737  0.00940896  0.00696456 -0.0370924
  -0.06512973  0.00746737 -0.02236725 -0